In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import gc
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import roc_auc_score

In [16]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisosONEHOT.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantesONEHOT.csv")
vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")
postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [18]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
no_postulaciones=pd.merge(no_postulaciones,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
no_postulaciones=pd.merge(no_postulaciones,avisos,on='idaviso')

no_postulaciones.head()

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Por Horas,tipo_de_trabajo_Primer empleo,tipo_de_trabajo_Teletrabajo,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior
0,1112430536,EexaOz,0,38.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1112430536,owaWw0Z,0,20.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,1112430536,PmGPRd1,0,19.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,1112430536,bO4q324,0,25.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,1112430536,MVd5dlr,0,30.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [19]:
postulaciones=postulaciones.append(no_postulaciones, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)

postulaciones = postulaciones.sample(len(postulaciones)//2)
print(len(postulaciones))

12793168
6393793


In [20]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.total_vistas=postulaciones.total_vistas.fillna(0)
no_postulaciones = 0
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Teletrabajo,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,cantidad,total_vistas
4751135,1112381227,Bm54m9R,0,20.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
3775102,1112361411,ak4q8oD,0,34.0,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.0,0.0
915842,1112281164,Nz02dQx,1,25.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,0.0,0.0
396317,1112414677,mzdLD96,1,27.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,13.0,33.0
1378321,1112370888,VNrok3z,0,25.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,0.0,0.0
3902939,1111780148,alQeW9,0,38.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
172604,1112417966,MVz9YRK,0,23.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
395636,1112306347,QNqeLX4,1,31.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
997232,1112278938,OqdNEZr,0,28.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
6320379,1112440376,NzrbOel,1,37.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,5.0,74.0


In [27]:
areas=avisos['nombre_area'].value_counts().index
postulaciones.drop(["nombre_area"],axis=1,inplace=True)
#postulaciones['nombre_area']=postulaciones['nombre_area'].replace(areas, range(len(areas)))
postulaciones.nombre_area

KeyError: "labels ['nombre_area'] not contained in axis"

In [22]:
train, test=train_test_split(postulaciones,test_size=0.10,random_state=int(time.time()))
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']


gc.collect()

Train:  5754413 Test:  639380


28

In [23]:
gnb = GaussianNB()
gnb.fit(x_train,y_train)

ValueError: could not convert string to float: 'Comercial'

In [ ]:
bnb = BernoulliNB(alpha=2.0)
bnb.fit(x_train,y_train)

In [ ]:
mnm = MultinomialNB()
mnm.fit(x_train,y_train)

In [ ]:
score=gnb.score(x_test,y_test)*100
print("GNB: ",score)
score=bnb.score(x_test,y_test)*100
print("BNB: ",score)
score=mnm.score(x_test,y_test)*100
print("MNM: ",score)


In [ ]:
def roc_score(model):
    x_final = x_test[features]
    y_final = model.predict_proba(x_final)
    y_pred = []
    for y in y_final:
        y_pred.append((y[1]))
    return roc_auc_score(y_test,y_pred)


In [ ]:
score=roc_score(gnb)*100
print("GNB: ",score)
score=roc_score(bnb)*100
print("BNB: ",score)
score=roc_score(mnm)*100
print("MNM: ",score)